# Helper functions

In [ ]:
def run_ml_capture_data(numFrames = 4000, initialDelay = 25.0, rate=1000):
    period = int((1/rate)/0.00000001)
    if os.path.isfile(f'{filename}'):   
        os.remove(f'{filename}')    
    print(filename)
    # Setup and open the file writer
    writer = root.dataWriter._writer
    writer.open(f'{filename}')

    # Wait for the file write to open the file
    while( writer.isOpen() is False):
        time.sleep(0.1)

    # Wait a little bit for the file to open up
    time.sleep(1.0)    

    #sets TriggerRegisters
    AcqCount = root.EpixHR.TriggerRegisters.AcqCount.get()
    DaqCount = root.EpixHR.TriggerRegisters.DaqCount.get()
    print("Aquisition Counter " + str(AcqCount))
    print("DAQ Counter " + str(DaqCount))
    root.EpixHR.TriggerRegisters.AutoTrigPeriod.set(period) #in steps of 10ns
    root.EpixHR.TriggerRegisters.AutoRunEn.set(True)
    root.EpixHR.TriggerRegisters.RunTriggerEnable.set(True)
    #wait for the camera to get thermally stable
    time.sleep(initialDelay)    
    root.EpixHR.TriggerRegisters.AutoDaqEn.set(True)
    root.EpixHR.TriggerRegisters.DaqTriggerEnable.set(True)
    initialFrameCounter = writer.getFrameCount()
    # Wait for the file write noise data
    while(writer.getFrameCount() < initialFrameCounter + numFrames):
        print( f'Number of frames received: {writer.getFrameCount()}' )
        time.sleep(1)

    # stops triggers
    root.EpixHR.TriggerRegisters.RunTriggerEnable.set(False)
    time.sleep(0.1)

    # Print the status
    DeltaAcqCount = root.EpixHR.TriggerRegisters.AcqCount.get() - AcqCount
    DeltaDaqCount = root.EpixHR.TriggerRegisters.DaqCount.get() - DaqCount
    print( f'Delta Aquisition Counter: {DeltaDaqCount}')
    print( f'Number of frames received: {writer.getFrameCount()}' )

    # Close the file writer
    writer.close()

    # Wait for the file write to close the file
    while( writer.isOpen() is True):
        time.sleep(0.1)

    ########################################################################################################################3
    f = open(filename, mode = 'rb')
    imgDesc = []
    for i in range(int(numFrames/MAX_NUMBER_OF_FRAMES_PER_BATCH)-1):
        print("Starting to get data set %d" % (i))
        allFrames = currentCam.getData(f,8)
        imgDesc2 = currentCam.getDescImaData(allFrames[:,:-8])
        if i == 0:
            headers = allFrames[:,0:6]
            imgDesc = imgDesc2
        if i == 1: #skips the first dataset
            headers = allFrames[:,0:6]
            imgDesc = imgDesc2
        else:
            headers = np.concatenate((headers, allFrames[:,0:6]),0)
            imgDesc = np.concatenate((imgDesc, imgDesc2),0)
        if allFrames.shape[0] != MAX_NUMBER_OF_FRAMES_PER_BATCH:
            break


    numberOfFrames = allFrames.shape[0]
    print(allFrames.shape)
    ###########################################################################################################################
    return imgDesc


In [ ]:
def read_dataset(numFrames = 4000):
    print(filename)
    # Setup and open the file writer
   
    ########################################################################################################################3
    f = open(filename, mode = 'rb')
    imgDesc = []
    for i in range(int(numFrames/MAX_NUMBER_OF_FRAMES_PER_BATCH)-1):
        print("Starting to get data set %d" % (i))
        allFrames = currentCam.getData(f,8)
        imgDesc2 = currentCam.getDescImaData(allFrames[:,:-8])
        if i == 0:
            headers = allFrames[:,0:6]
            imgDesc = imgDesc2
        if i == 1: #skips the first dataset
            headers = allFrames[:,0:6]
            imgDesc = imgDesc2
        else:
            headers = np.concatenate((headers, allFrames[:,0:6]),0)
            imgDesc = np.concatenate((imgDesc, imgDesc2),0)
        if allFrames.shape[0] != MAX_NUMBER_OF_FRAMES_PER_BATCH:
            break


    numberOfFrames = allFrames.shape[0]
    print(allFrames.shape)
    ###########################################################################################################################
    return imgDesc

In [ ]:
def show_random_images(this_dataset, dataset_type):
    # create figure 
    fig = plt.figure(figsize=(10, 4)) 
  
    # setting values to rows and column variables 
    rows = 2
    columns = 2

    for idx in range(4):

        # Adds a subplot at the 1st position 
        fig.add_subplot(rows, columns, idx+1) 

        # randomly selects an image
        randomIdx =np.random.randint(0,this_dataset.shape[0])

        # showing image 
        plt.imshow(this_dataset[randomIdx]) 
        plt.axis('off') 
        plt.title(f'Dataset_%s Image %d' % (dataset_type, randomIdx)) 